In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MaxAbsScaler
import matplotlib.pyplot as plt
import seaborn as sns
import re
color = sns.color_palette()

%matplotlib inline

pd.options.mode.chained_assignment = None
pd.options.display.max_columns = 999
from subprocess import check_output

import warnings

def ignore_warn(*args ,**kwargs):
    pass
warnings.warn = ignore_warn

In [1125]:
train = pd.read_csv("./data/train_20171226.csv")
test = pd.read_csv("./data/yancheng_testB_20180224.csv")
print(train.shape,test.shape)

(20157, 32) (140, 3)


In [1126]:
#获取特征sale_month, sale_year
train['sale_month'] = train['sale_date']%100
train['sale_year'] = train['sale_date']//100%100
test['sale_month'] = test['predict_date']%100
test['sale_year'] = test['predict_date']//100%100

In [1127]:
#merge train dataset and fill test dataset
merge_train = pd.DataFrame()
all_class_id = train['class_id'].unique().tolist()
for idx in all_class_id:
    all_month = train['sale_date'][train['class_id']==idx].unique().tolist()
    for mon in all_month:
        max_val = train['sale_quantity'][train['class_id']==idx][train['sale_date']==mon].max()
        sale_sum = train['sale_quantity'][train['class_id']==idx][train['sale_date']==mon].sum()
        feat_val = train[train['class_id']==idx][train['sale_date']==mon][train['sale_quantity']==max_val]
        feat_val['sale_quantity'] = sale_sum
        merge_train = pd.concat([merge_train, feat_val[0:1]])
#含有相同销量的同一class_id
merge_train = merge_train.sort_values(by=['sale_date'], ascending=True)

In [1128]:
merge_train = merge_train[['sale_date', 'class_id', 'sale_quantity', 'sale_month', 'sale_year']]
cols = ['sale_date', 'class_id', 'sale_quantity', 'sale_month', 'sale_year']
###############去除11月份 ##################
merge_train = merge_train[merge_train['sale_month']!=11]
test = test.ix[:,cols]
test['sale_date'] = test['sale_date'].fillna(201712)
test['sale_date'] = test['sale_date'].astype(int)

In [1129]:
count = merge_train[['class_id','sale_quantity']].groupby(['class_id'], as_index=1).count().sort_values(['sale_quantity'])
print('less than 10:',count[count['sale_quantity']<14].count())

less than 10: sale_quantity    34
dtype: int64


In [1130]:
print(merge_train.shape, test.shape)
class_12 = count[count['sale_quantity']<14]
class_12 = class_12.index.tolist()
all_data = pd.concat((merge_train, test)).reset_index(drop=True)
for idx in class_12:
    li = []
    li = all_data[all_data['class_id']==idx].index.tolist()
    all_data.drop(all_data.index[li],inplace=True)
    all_data = all_data.reset_index(drop=True)

(5195, 5) (140, 5)


In [1191]:
def rolling(df,roll,window):
    last=roll(df,window=window).tolist()
    last=[np.NaN]+last
    last.pop()
    return last

new = pd.DataFrame()
for a in all_class_id:
    df = all_data[all_data['class_id'] == a]
    df['last_1_sum'] = rolling(df['sale_quantity'], pd.rolling_sum, window=1)
    df['last_2_sum'] = rolling(df['sale_quantity'], pd.rolling_sum, window=2)
    df['last_3_sum'] = rolling(df['sale_quantity'], pd.rolling_sum, window=3)
    df['last_4_sum'] = rolling(df['sale_quantity'], pd.rolling_sum, window=4)
    df['last_5_sum'] = rolling(df['sale_quantity'], pd.rolling_sum, window=5)
    df['last_6_sum'] = rolling(df['sale_quantity'], pd.rolling_sum, window=6)
    df['last_7_sum'] = rolling(df['sale_quantity'], pd.rolling_sum, window=7)
    df['last_8_sum'] = rolling(df['sale_quantity'], pd.rolling_sum, window=8)
    df['last_9_sum'] = rolling(df['sale_quantity'], pd.rolling_sum, window=9)
    df['last_10_sum'] = rolling(df['sale_quantity'], pd.rolling_sum, window=10)
    df['last_11_sum'] = rolling(df['sale_quantity'], pd.rolling_sum, window=11)
    df['last_12_sum'] = rolling(df['sale_quantity'], pd.rolling_sum, window=12)
    df['last_13_sum'] = rolling(df['sale_quantity'], pd.rolling_sum, window=13)
    
    df['last_5_mean'] = rolling(df['sale_quantity'], pd.rolling_mean, window=5)
    
    df['last_5_std'] = rolling(df['sale_quantity'], pd.rolling_std, window=5)
    
    df['move_1'] = df['last_1_sum']
    df['move_2'] = df['last_2_sum'] - df['last_1_sum']
    df['move_3'] = df['last_3_sum'] - df['last_2_sum']
    df['move_4'] = df['last_4_sum'] - df['last_3_sum']
    df['move_5'] = df['last_5_sum'] - df['last_4_sum']
    
    df['move_9'] = df['last_9_sum'] - df['last_8_sum']
    
    df['move_11'] = df['last_11_sum'] - df['last_10_sum']
    df['move_12'] = df['last_12_sum'] - df['last_11_sum']
    df['move_13'] = df['last_13_sum'] - df['last_12_sum']
    
    
    df['diff_12'] = df['move_11'] - df['move_12']
    #df['diff_13'] = df['move_11'] - df['move_13']
    
    #对于12月份的滑动值特殊处理,因为不存才201711的特征
    #diff = df['move_11'][df['sale_month']==12] - df['move_13'][df['sale_month']==12]
    #df['diff_12'][df['sale_month']==12] = diff
    #move = df['last_11_sum'][df['sale_month']==12] - df['last_10_sum'][df['sale_month']==12]
    #df['move_12'][df['sale_month']==12] = move
    
    new = pd.concat([new, df])

In [1192]:
data = new.drop(['last_1_sum', 'last_2_sum', 'last_3_sum', 'last_4_sum', 'last_5_sum',
                 'last_6_sum', 'last_7_sum', 'last_8_sum', 'last_9_sum', 'last_10_sum',
                 'last_11_sum', 'last_12_sum', 'last_13_sum', 'move_12', 'move_13'], axis=1)
test = data[data['sale_quantity'].isnull()]
train_X = data.dropna()

In [1193]:
#spilt dataset
val_data = train_X[train_X['sale_date']==201710]
val_data = val_data.drop(['sale_date'], axis=1)

test = test.drop(['sale_date', 'sale_quantity'], axis=1)

#训练集
train_x = train_X[train_X['sale_date']!=201710]
train_y = train_x['sale_quantity']
train_x = train_x.drop(['sale_date', 'sale_quantity'], axis=1)

#最终训练数据集
y = train_X['sale_quantity']
train_X = train_X.drop(['sale_date', 'sale_quantity'], axis=1)

print(train_X.shape, train_x.shape, train_y.shape, test.shape, len(train_x['class_id'].unique().tolist()),len(test['class_id'].unique().tolist()))

(3656, 13) (3550, 13) (3550,) (106, 13) 106 106


In [1194]:
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import ElasticNet,Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from mlxtend.regressor import StackingRegressor
import xgboost as xgb
import lightgbm as lgb

In [1195]:
def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

def xgb_Regressor(train_x, train_y, val_data, train_X, y):
    xgb_model = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=1200,
                             reg_alpha=0.1640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)
    xgb_model.fit(train_x, train_y)
    pred_val = xgb_model.predict(val_data.drop(['sale_quantity'],axis=1))
    score = rmsle(val_data['sale_quantity'], pred_val)
    xgb_model.fit(train_X, y)
    
    return xgb_model, score, pred_val

def lgb_Regressor(train_x, train_y, val_data, train_X, y):
    lgb_model = lgb.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.05, n_estimators=360,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 10)
    lgb_model.fit(train_x, train_y)
    pred_val = lgb_model.predict(val_data.drop(['sale_quantity'],axis=1))
    score = rmsle(val_data['sale_quantity'], pred_val)
    lgb_model.fit(train_X, y)
    
    return lgb_model, score, pred_val
def base_model():
    ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=3))
    lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=1))

    return ENet,lasso

def gboost_Regressor(train_x, train_y, val_data, train_X, y):
    gb_model = GradientBoostingRegressor(n_estimators=3600, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=20, min_samples_split=20, 
                    loss='huber', random_state =5)
    gb_model.fit(train_x, train_y)
    pred_val = gb_model.predict(val_data.drop(['sale_quantity'],axis=1))
    score = rmsle(val_data['sale_quantity'], pred_val)
    gb_model.fit(train_X, y)
    
    return gb_model, score, pred_val

In [1196]:
print("XGBOOSTRegressor开始训练...")
xgb_model, score, xgb_train_pred = xgb_Regressor(train_x, train_y, val_data, train_X, y)
print(score)
xgb_pred = xgb_model.predict(test)

XGBOOSTRegressor开始训练...
152.774456446


In [1197]:
print("LGBMRegressor开始训练...")
lgb_model,score, lgb_train_pred= lgb_Regressor(train_x, train_y, val_data, train_X, y)
print(score)
lgb_pred = lgb_model.predict(test)

LGBMRegressor开始训练...
130.382680099


In [1198]:
print("GDBTRegressor开始训练...")
gb_model,score, gb_train_pred = gboost_Regressor(train_x, train_y, val_data, train_X, y)
print( score)
gb_pred = gb_model.predict(test)

GDBTRegressor开始训练...
118.19399117


In [1199]:
ENet,lasso= base_model()
    
ENet.fit(train_X, y)
enet_pred = ENet.predict(val_data.drop(['sale_quantity'],axis=1))
print("ENet:", rmsle(val_data['sale_quantity'], enet_pred))
    
lasso.fit(train_X, y)
lasso_pred = lasso.predict(val_data.drop(['sale_quantity'],axis=1))
print("lasso:", rmsle(val_data['sale_quantity'], lasso_pred))

ENet: 144.314851514
lasso: 144.318756857


In [1200]:
# ####Stacking####
print('Stacking...')
stacked_averaged_models = StackingRegressor(
regressors=[ENet,lasso, lgb_model, gb_model],
meta_regressor= xgb_model
)
stacked_averaged_models.fit(train_x, train_y)
stacked_train_pred = stacked_averaged_models.predict(val_data.drop(['sale_quantity'],axis=1))
stacked_averaged_models.fit(train_X, y)
stacked_pred = stacked_averaged_models.predict(test.values)
print(rmsle(val_data['sale_quantity'], stacked_train_pred))
print(rmsle(val_data['sale_quantity'], stacked_train_pred*0.30 + gb_train_pred*0.30 + 
       lgb_train_pred*0.20 + xgb_train_pred*0.20))
ensemble = stacked_pred*0.30 + gb_pred*0.30  + lgb_pred*0.20 + xgb_pred*0.20

Stacking...
114.153556252
117.934953099


In [1204]:
#submission
test['sale_quantity'] = ensemble
cols = test['class_id'].unique().tolist()
sub = pd.read_csv("./data/yancheng_testB_20180224.csv")
for idx in cols:
    sale = test['sale_quantity'][test['class_id']==idx]
    sub['predict_quantity'][sub['class_id']==idx] = list(sale)[0]
for idx in class_12:
    sale = merge_train['sale_quantity'][merge_train['class_id']==idx][merge_train['sale_date']==201710]
    sub['predict_quantity'][sub['class_id']==idx] = list(sale)[0]*1.3
sub.to_csv('submission_hejian_27.csv',index=False)

In [1203]:
test.shape

(106, 13)

In [2]:
test = pd.read_csv("first_test_index_20180131.csv")
test.head()

,id
0,822405
1,1142964
2,1241977
3,1309049
4,1085810


In [4]:
test['type'] = 'galaxy'
test.to_csv("tianwen_galaxy.csv", index=False, header=False)
test['type'] = 'qso'
test.to_csv("tianwen_qso.csv", index=False, header=False)